In [34]:
import pandas as pd
from pandas import DataFrame as df
import xml.etree.ElementTree as ET
import numpy as np
import re

In [35]:
DATE_TEMPLATE = re.compile(r'\d{2}\.\d{2}\.\d{4}')

In [108]:
tree = ET.parse('output.xml')
root = tree.getroot()
r = root.findall("Lead")

def make_dict_of_leads(root=root):
    dict_of_leads = dict();
    for lead in root.iter('Lead'):
        id = lead.attrib["id"]

        pulpy = ET.fromstring(lead.attrib["text"])
        source_sentence = pulpy.find("b").find("s")

        remove_explicit_from_sentence(source_sentence)

        text = ''.join(source_sentence.itertext())
        
        
    
        dict_of_leads[id] = text;
        
        
    return dict_of_leads;

def remove_explicit_from_sentence(xml_sentence):
    # to remove
    phoneno = re.compile(r'\d{11}')

    for i in xml_sentence:
        if re.match(phoneno, i.attrib["lemma"]):
            xml_sentence.remove(i)
    

make_dict_of_leads()["46"]

one = make_dict_of_leads()['0']


"""
   <b>
      <s>
         21.06.2016
         <P n0="" lemma="89140580517">89140580517</P>
         хочет
         <W n1="" lemma="buy">купить</W>
         <W n2="" lemma="ноутбук">ноутбук</W>
         за 17000 руб ездить
         <P n3="" lemma="Якутия">Якутия</P>
         .
      </s>
   </b>
"""


one

'21.06.2016 купить ноутбук за 17000 руб ездить Якутия.'

In [38]:
# todo: помнить о тексте лида. Там выделены факты прямо в разметке - полезно при выводе информации в веб-интерфейсе

def compare_facts_to_leads(root=root):
    facts_grouped_by_lead = dict()

    for i in root.find("document").find('facts'):
        lead_id = i.attrib['LeadID']
        if facts_grouped_by_lead.get(lead_id):
            facts_grouped_by_lead[lead_id].append(i)
        else:
            facts_grouped_by_lead[lead_id] = [i]
    return facts_grouped_by_lead

def make_common_table():    
    appendix = [
        "date",
        "conversation",
        "source_sentence"
    ]
    calls = df(columns=appendix)
    
    facts = compare_facts_to_leads()
    leads = make_dict_of_leads()
    for lead in facts:
        elems = facts[lead]
        print(leads[lead])
        one_sentence = lead
        cols = []
        values = []
        for fact_name in elems:
            for fact_field in fact_name:
                cols.append(fact_name.tag + "_" + fact_field.tag)
                values.append(fact_field.attrib["val"])
        one_row = pd.DataFrame([values], columns=cols)
        calls = calls.append(one_row)

        values = []
        cols = []
    return calls;

calls = make_common_table()


to_str = lambda x: " " + x if x is not np.nan else ""
calls["notebook"] = calls["Notebook_Word"] + calls["Notebook_Vendor"].map(to_str)
calls["phone"] = calls["Phone_Word"] + calls["Phone_Vendor"].map(to_str) + calls["Phone_Model"].map(to_str)

calls.drop(['Notebook_Word', 'Notebook_Vendor'], axis=1, inplace=True)
calls.drop(['Phone_Word', 'Phone_Vendor', 'Phone_Model'], axis=1, inplace=True)


pawn = calls[calls.Pawn_Word.notnull()]
buy_out = calls[calls.Buy_Word.notnull()]
buy_out.to_excel("buy_out.xlsx")
pawn.to_excel("pawn.xlsx")

calls.to_excel("whole_table.xlsx")

27.06.2016 89140580517 спр адрес наш.
24.06.2016 89140580517 продажа т samsung a5 на выезде Ляпидевского 12.
28.06.2016 89140580517 неадекват.
21.06.2016 89140580517 куп тел 27.
22.06.2016 89140580517 куп 12 выезд Широких-Полянского 27, Виталий.
23.06.2016 89140580517 s Acer V5 17 зац.
24.06.2016 89140580517 продажа тел samsung на выезде Ляпидевского 12.
29.06.2016 89140580517 ватсап ломбард ноут цена оператора 4000.
24.06.2016 89140580517 продажа т samsung а5 на выезде Ляпидевского 12.
24.06.2016 89140580517 продажа айфон6s на выезде Ляпидевского 12.
23.06.2016 89140580517 прод Асер офисный предложил 3000 зацепка.
21.06.2016 89140580517 куп 12.
24.06.2016 89140580517 продажа тел айфон 6s на выезде Ляпидевского 12.
23.06.2016 89140580517 ломб Асер 5733 наша цена 5000.
24.06.2016 89140580517 продажа samsung на выезде Ляпидевского 12.
21.06.2016 89140580517 куп 30.
29.06.2016 89140580517 ватсапп тв продать зацепка.
22.06.2016 89140580517 покупает 17 зац.
21.06.2016 89140580517 уу ноутбук

AttributeError: 'DataFrame' object has no attribute 'Buy_Word'

In [22]:
date_string = ""

f = open("manual_input.txt", "r", encoding="utf8")
preprocessed_input = open("preprocessed_input.txt", "w", encoding="utf8")
ff = f.read().splitlines()

for i in ff:
    if re.match(DATE_TEMPLATE, i) is not None:
        date_string = i
    else:
        preprocessed_input.write(date_string + " " + i + "\n")
f.close()
preprocessed_input.close()
